In [54]:
import pandas as pd
import json
import numpy as np
import os
from IPython import embed

In [55]:
for attack in ["deepwordbug", "textfooler", "textbugger"]:
    files = [file for file in os.listdir() if file.startswith(f"adv_{attack}")]
    dfs = [pd.read_csv(file) for file in files]
    all_original_texts = [df["original_text"].values for df in dfs]
    all_perturbed_texts = [df["perturbed_text"].values for df in dfs]
    all_labels = [df["label"].values for df in dfs]

    x1 = all_original_texts[0]
    x1_clean = [i.replace("[", "").replace("]", "") for i in x1]
    x1_clean_to_x1 = {i: j for i, j in zip(x1_clean, x1)}
    y1 = [x.replace("[", "").replace("]", "").split() for x in all_perturbed_texts[0]]
    x1_clean_to_y1 = {i: j for i, j in zip(x1_clean, y1)}
    labels1 = all_labels[0]
    x1_clean_to_labels1 = {i: j for i, j in zip(x1_clean, labels1)}

    x2 = all_original_texts[1]
    x2_clean = [i.replace("[", "").replace("]", "") for i in x2]
    x2_clean_to_x2 = {i: j for i, j in zip(x2_clean, x2)}
    y2 = [x.replace("[", "").replace("]", "").split() for x in all_perturbed_texts[1]]
    x2_clean_to_y2 = {i: j for i, j in zip(x2_clean, y2)}

    x3 = all_original_texts[2]
    x3_clean = [i.replace("[", "").replace("]", "") for i in x3]
    x3_clean_to_x3 = {i: j for i, j in zip(x3_clean, x3)}
    y3 = [x.replace("[", "").replace("]", "").split() for x in all_perturbed_texts[2]]
    x3_clean_to_y3 = {i: j for i, j in zip(x3_clean, y3)}

    x_clean_commons = set.intersection(set(x1_clean), set(x2_clean), set(x3_clean))
    print(len(x_clean_commons))

    results_original_texts = []
    results_perturbed_texts = []
    results_labels = []

    for x_clean in x_clean_commons:
        x_clean_splited = x_clean.split()
        a = x1_clean_to_x1[x_clean]
        b = x2_clean_to_x2[x_clean]
        c = x3_clean_to_x3[x_clean]

        a_ = x1_clean_to_y1[x_clean]
        b_ = x2_clean_to_y2[x_clean]
        c_ = x3_clean_to_y3[x_clean]

        a_parts_to_replace = [
            i
            for i, text in enumerate(a.split())
            if text.startswith("[[") and text.endswith("]]")
        ]
        b_parts_to_replace = [
            i
            for i, text in enumerate(b.split())
            if text.startswith("[[") and text.endswith("]]")
        ]
        c_parts_to_replace = [
            i
            for i, text in enumerate(c.split())
            if text.startswith("[[") and text.endswith("]]")
        ]

        resulted_text_parts = []

        for i in range(len(x_clean.split())):
            if i in a_parts_to_replace:
                # print("a", a_[i])
                resulted_text_parts.append(a_[i])
            elif i in b_parts_to_replace:
                # print("b", b_[i])
                resulted_text_parts.append(b_[i])
            elif i in c_parts_to_replace:
                # print("c", c_[i])
                resulted_text_parts.append(c_[i])
            else:
                # print("-", x_clean_splited[i])
                resulted_text_parts.append(x_clean_splited[i])
        resulted_text = " ".join(resulted_text_parts)
        results_original_texts.append(x_clean)
        results_perturbed_texts.append(resulted_text)
        results_labels.append(x1_clean_to_labels1[x_clean])
    pd.DataFrame(
        {
            "original_text": results_original_texts,
            "perturbed_text": results_perturbed_texts,
            "label": results_labels,
        }
    ).to_csv(f"adv_{attack}_combined.csv", index=False)

408
631
461
